In [33]:
import keras
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D

from keras.applications.resnet import ResNet50, ResNet101, preprocess_input

import tensorflow as tf
import numpy as np


In [13]:
cnn_pretrained = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

What is the mnist image size?

In [ ]:
cnn = Sequential()
cnn.add(cnn_pretrained)
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(10, activation='softmax'))

# do with functional interface

Do feature extraction

In [25]:
img_size = (224, 224)

cnn_pretrained = ResNet101(weights='imagenet', include_top=False, input_shape=img_size + (3,))

In [26]:
dataset = keras.utils.image_dataset_from_directory(
    "dataset",
    labels = "inferred",
    batch_size = 32,
    image_size = img_size,
    color_mode = "rgb",
    interpolation = "bilinear"
)

Found 11 files belonging to 2 classes.


In [29]:
def preprocess_batch(batch_images, batch_labels):
    batch_images = preprocess_input(batch_images)  # This applies ImageNet normalization
    return batch_images, batch_labels

dataset = dataset.map(preprocess_batch)

In [34]:
all_images = []
all_labels = []

# Iterate through the dataset to collect all images and labels
for images, labels in dataset:
    all_images.append(images.numpy())
    all_labels.append(labels.numpy())

all_images = np.concatenate(all_images, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

In [30]:
predictions = cnn_pretrained.predict(dataset)

2025-03-19 02:09:27.220550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [11]
	 [[{{node Placeholder/_4}}]]
2025-03-19 02:09:27.220851: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [11]
	 [[{{node Placeholder/_4}}]]


1/1 [==============================] - 3s 3s/step


In [31]:
predictions.shape

(11, 7, 7, 2048)

In [36]:
predictions = predictions.reshape(predictions.shape[0], -1)
predictions.shape

(11, 100352)

In [37]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(predictions, all_labels)

SVC()

In [38]:
test_image = keras.utils.load_img(
    "extra_dataset/ginger-cat.jpg", target_size=img_size
)
test_image = keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = preprocess_input(test_image)

prediction = cnn_pretrained.predict(test_image)
prediction = prediction.reshape(prediction.shape[0], -1)
print("Predicted class:", svc.predict(prediction)[0])
print("Actual class:  1")

1/1 [==============================] - 2s 2s/step
Predicted class: 0
Actual class:  1


In [39]:
test_image = keras.utils.load_img(
    "extra_dataset/golden-retriever.jpg", target_size=img_size
)
test_image = keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = preprocess_input(test_image)

prediction = cnn_pretrained.predict(test_image)
prediction = prediction.reshape(prediction.shape[0], -1)
print("Predicted class:", svc.predict(prediction)[0])
print("Actual class:  1")

1/1 [==============================] - 0s 367ms/step
Predicted class: 0
Actual class:  1
